In [1]:
# %%
# Learning point:
# Working with RGB images and pretrained models
# Using a pretrained model for transfer learning
# Task:
# Train a ResNet18 model on the CIFAR10 dataset
# Learn how to add data augmentation to the transformation pipeline

# Note: MLP will struggle with CIFAR10 dataset due to its complexity
# Note: CIFAR10 dataset has 3 channels (RGB) and 10 classes
# Note: ResNet18 is a popular model for image classification tasks
# Note: CIFAR10 dataset has 60,000 32x32 color images in 10 classes, with 6,000 images per class
# Note: The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.

import numpy as np
from sklearn.metrics import classification_report

import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torchvision.models as models

torch.manual_seed(42)

In [2]:
# %%
device = torch.device("cpu")

if torch.backends.mps.is_available():
    device = torch.device("mps")
if torch.cuda.is_available():
    device = torch.device("cuda")

print(f"Using device: {device}")
# Model pretrained on imagenet

Using device: mps


In [3]:
# %%
resnet18 = models.resnet18(pretrained=True).to(device)
fc = nn.Linear(1000, 10)
model = nn.Sequential(resnet18, fc)

/Users/ctr26/Documents/2024_ebi_course/pytorch_tutorial/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ctr26/Documents/2024_ebi_course/pytorch_tutorial/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# %%
# Larger transformation pipeline for imagenet
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

In [5]:
# %%
train_data = datasets.CIFAR10(
    root="./data/CIFAR10", train=True, transform=transform, download=True
)
test_data = datasets.CIFAR10(
    root="./data/CIFAR10", train=False, transform=transform, download=True
)

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(
    test_data,
    batch_size=64,
    shuffle=True,
)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# %%

learning_rate = 1e-3
epochs = 100
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [7]:
# %%
model = model.to(device)

for epoch in tqdm(range(epochs)):
    size = len(train_dataloader.dataset)
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        # loss = loss_fn(y, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Epoch: {epoch+1}, Loss: {loss:.6f}, Progress: [{current}/{size}]")


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1, Loss: 2.420524, Progress: [0/50000]
Epoch: 1, Loss: 1.188298, Progress: [6400/50000]


In [ ]:
# %%
# Test the model cross validation


model.eval()
y_pred = []
y_true = []

In [ ]:
# %%
# Disable gradient computation for evaluation to save memory and computations
with torch.no_grad():
    all_preds = []
    all_labels = []

    for X, y in test_dataloader:
        X = X.device()  # No device assignment since it defaults to CPU in your script
        preds = model(X).to(device).cpu()
        all_preds.extend(preds.argmax(1).numpy())  # Get the predicted classes
        all_labels.extend(y.numpy())

# Convert list to NumPy arrays for Scikit-Learn
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# Classification report
report = classification_report(
    all_labels, all_preds, target_names=[str(i) for i in range(10)]
)


print("Classification Report:\n", report)
